# <b>Getting started with Generative-AI </b>
Submitted By: <b><i>Jenish Twayana</i></b>  

Submitted Date: <b><i>4th September, 2024</i></b>  

## Exercise 3: Building a Sequential Chain
<b>Objective:</b>    

Create a sequential chain in Langchain that performs the following operations on a given input
text:
● Convert the text to lowercase.
● Remove any punctuation from the text.
● Identify and extract named entities from the text (e.g., names of people, organizations,
locations).
● Summarize the text to one sentence.
● Return the processed text, the named entities, and the summary.

<b>Text:</b>  

"Apple Inc. is an American multinational technology company headquartered in Cupertino,
California, that designs, develops, and sells consumer electronics, computer software, and
online services. It is considered one of the Big Five companies in the U.S. information
technology industry, along with Amazon, Google, Microsoft, and Facebook."

### Import the necessary packages and the libraries

In [87]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough
import os

### Set up the environment variables for LangChain Tracing
It logs the tracing data in the LangSmith web interface.

In [2]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]="assignment-3"

### Initialise the Chat Model

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

### Input Text

In [145]:
text = '''Apple Inc. is an American multinational technology company headquartered in Cupertino, 
California, that designs, develops, and sells consumer electronics, computer software, and online services. 
It is considered one of the Big Five companies in the U.S. information technology industry, along with Amazon, Google, Microsoft, and Facebook.'''

### Create a custom chain to convert the text into lowercase

In [146]:
def to_lowercase(input: str) -> dict:
    return {"text": input, "lowercase_text": input.lower()}

lowercase_text_chain = RunnableLambda(to_lowercase) # converts callable into runnable

### Create a chain that removes the punctuations from the text

In [147]:
punctuation_remove_template: str = """ 
Remove the punctuation from the following text:
{text}
"""

punctuation_remove_prompt_template = ChatPromptTemplate.from_template(
    punctuation_remove_template)

punctuation_remove_chain = punctuation_remove_prompt_template | llm | StrOutputParser()

### Create a chain that extracts the entities from the text

In [168]:
extract_entities_template: str = """ 
Identify and extract named entities from the following text. The entities can be names of people, organizations or
locations etc. The desired output is valid python JSON format where the key is the entity category and values are the list of entities.
Here is the text:
{text}
"""

extract_entities_prompt_template = ChatPromptTemplate.from_template(
    extract_entities_template)

extract_entities_chain = extract_entities_prompt_template | llm | StrOutputParser()

### Create a chain that summarizes the text

In [156]:
summarize_text_template: str = """ 
Summarize the following text in one sentence:
{text}
"""

summarize_text_prompt_template = ChatPromptTemplate.from_template(
   summarize_text_template)

summarize_text_chain = summarize_text_prompt_template | llm | StrOutputParser()

### Create a sequential chain

In [169]:
ss_chain = (lowercase_text_chain | 
            RunnablePassthrough.assign(punctuation_removed_text=punctuation_remove_chain) |  # runnablepassthrough is used to store the output of chain
            RunnablePassthrough.assign(extracted_entities=extract_entities_chain) | 
            RunnablePassthrough.assign(summarized_text=summarize_text_chain))

output = ss_chain.invoke(text)
print(output)

### Display the output with proper formatting

In [170]:
for k, v in output.items():
    print(f"{k}: {v}")
    print("\n")

text: Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services. It is considered one of the Big Five companies in the U.S. information technology industry, along with Amazon, Google, Microsoft, and Facebook.


lowercase_text: apple inc. is an american multinational technology company headquartered in cupertino, california, that designs, develops, and sells consumer electronics, computer software, and online services. it is considered one of the big five companies in the u.s. information technology industry, along with amazon, google, microsoft, and facebook.


punctuation_removed_text: Apple Inc is an American multinational technology company headquartered in Cupertino California that designs develops and sells consumer electronics computer software and online services It is considered one of the Big Five companies in the US information technology ind